In [4]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os

# --- Configuration ---
# Specify the name of your input Excel file
input_filename = 'train_data.xlsx' # <--- Updated to your filename
# Specify the name for the output Excel file
output_filename = 'train_data_900k.xlsx'
# Specify the target number of rows
target_num_rows = 900000
# Specify the duration in years for generating future dates
# Data will be spread across this many years starting from the day after the last original date.
# Increase this value if you want fewer entries per future day, or decrease for more.
future_years_to_generate = 5 # <--- You can adjust this value

# Assumed column names based on the likely header row and your previous error.
# If your actual header row has different names, update these strings.
date_col = 'Date'
country_col = 'Country'
city_col = 'City'
shop_col = 'Shop'
target_col = 'Target'

# List of columns for location combinations
location_cols = [country_col, city_col, shop_col]

# --- Data Processing ---
try:
    # Read the original Excel file - assuming it has a header row now
    df_original = pd.read_excel(input_filename)

    # --- Validate Columns ---
    # Check if the expected columns are present after reading with header
    required_cols = [date_col, country_col, city_col, shop_col, target_col]
    if not all(col in df_original.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_original.columns]
        raise KeyError(f"The following required columns were not found in the Excel file based on the header: {missing}. Please check the '{required_cols}' list in the script and the header row of your file.")

    # Get the current number of rows (excluding header)
    current_num_rows = len(df_original)

    if current_num_rows == 0:
        print(f"Error: The file '{input_filename}' is empty or contains no data after reading the header.")
    elif current_num_rows >= target_num_rows:
        print(f"The original file already contains {current_num_rows} rows, which is equal to or more than the target ({target_num_rows}). No expansion needed.")
        # Optionally save the original file to the output name if it meets the criteria
        # df_original.to_excel(output_filename, index=False)
    else:
        print(f"Read {current_num_rows} rows from '{input_filename}'.")
        print(f"Generating data to reach {target_num_rows} rows, preserving original shop-location combinations and spreading over {future_years_to_generate} future years...")

        # Convert the 'Date' column to datetime objects
        df_original[date_col] = pd.to_datetime(df_original[date_col])

        # Find the last date in the original data
        last_date = df_original[date_col].max()

        # Calculate the number of rows to generate
        num_rows_to_generate = target_num_rows - current_num_rows

        # --- Generate New Data ---

        # 1. Generate Dates: Create a pool of dates for the specified number of future years
        start_date_for_generation = last_date + timedelta(days=1)
        end_date_for_generation = start_date_for_generation + timedelta(days=int(future_years_to_generate * 365.25)) # Approx days in future years

        # Generate all dates within this range
        future_date_range = pd.date_range(start=start_date_for_generation, end=end_date_for_generation, freq='D')

        # Convert to a list for easier cycling
        future_date_pool = future_date_range.tolist()
        num_dates_in_pool = len(future_date_pool)

        if num_dates_in_pool == 0:
             print("Error: Could not generate any dates in the specified future range. Check last date and future_years_to_generate.")
             exit()

        # 2. Generate Country, City, Shop: Cycle through UNIQUE (Country, City, Shop) tuples
        unique_shop_location_tuples = df_original[location_cols].drop_duplicates().values.tolist()
        num_unique_combinations = len(unique_shop_location_tuples)

        if num_unique_combinations == 0:
             print("Error: Could not find any unique Country, City, Shop combinations in the original data.")
             exit()

        new_shop_locations = []
        new_dates = []
        # Generate location and date pairs by cycling
        for i in range(num_rows_to_generate):
            location_index = i % num_unique_combinations
            date_index = i % num_dates_in_pool

            new_shop_locations.append(unique_shop_location_tuples[location_index])
            new_dates.append(future_date_pool[date_index])


        # Separate the new locations into lists for each column
        new_countries = [loc[0] for loc in new_shop_locations]
        new_cities = [loc[1] for loc in new_shop_locations]
        new_shops = [loc[2] for loc in new_shop_locations]


        # 3. Generate Target: Use statistics from the original data
        mean_target = df_original[target_col].mean()
        std_dev_target = df_original[target_col].std()

        # Generate random values from a normal distribution
        new_targets = np.random.normal(loc=mean_target, scale=std_dev_target, size=num_rows_to_generate)
        new_targets = np.maximum(0, np.round(new_targets)).astype(int) # Round and clip negative values

        # Create a new DataFrame with the generated data
        df_generated = pd.DataFrame({
            date_col: new_dates,
            country_col: new_countries,
            city_col: new_cities,
            shop_col: new_shops,
            target_col: new_targets
        })

        # Concatenate the original DataFrame and the generated DataFrame
        df_expanded = pd.concat([df_original, df_generated], ignore_index=True)

        # Ensure the total number of rows is exactly the target (safety check)
        if len(df_expanded) > target_num_rows:
             print(f"Warning: Final row count ({len(df_expanded)}) exceeds target ({target_num_rows}). Truncating.")
             df_expanded = df_expanded.head(target_num_rows)
        elif len(df_expanded) < target_num_rows:
             print(f"Warning: Final row count ({len(df_expanded)}) is less than target ({target_num_rows}). This is unexpected.")


        # Save the result to a new Excel file
        print(f"Saving expanded data ({len(df_expanded)} rows) to '{output_filename}'...")
        df_expanded.to_excel(output_filename, index=False)

        print(f"Successfully expanded the table with original shop-location combinations and saved to '{output_filename}'.")

except FileNotFoundError:
    print(f"Error: The file '{input_filename}' was not found. Please ensure the file is in the same directory as the script, or provide the full path.")
except KeyError as e:
    print(f"Error: Missing expected column(s). {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Read 1461 rows from 'train_data.xlsx'.
Generating data to reach 900000 rows, preserving original shop-location combinations and spreading over 5 future years...
Saving expanded data (900000 rows) to 'train_data_900k.xlsx'...
Successfully expanded the table with original shop-location combinations and saved to 'train_data_900k.xlsx'.


In [1]:
import modin.pandas as pd

data = pd.read_excel('train_data_900k.xlsx')

2025-05-15 09:35:01,806	INFO worker.py:1819 -- Started a local Ray instance.


In [2]:
print(data)

             Date Country        City    Shop  Target
0      2020-01-01  Canada     Toronto  Shop F     456
1      2020-01-02  Canada   Vancouver  Shop H     476
2      2020-01-03      UK  Manchester  Shop L     162
3      2020-01-04  Canada   Vancouver  Shop H     307
4      2020-01-05  Canada    Montreal  Shop I     262
...           ...     ...         ...     ...     ...
899995 2028-01-17  Canada     Toronto  Shop F     348
899996 2028-01-18  Canada   Vancouver  Shop H     305
899997 2028-01-19      UK  Manchester  Shop L     107
899998 2028-01-20  Canada    Montreal  Shop I     112
899999 2028-01-21     USA    New York  Shop A     168

[900000 rows x 5 columns]


Installing collected packages: webencodings, sortedcontainers, fastjsonschema, zict, widgetsnbextension, websocket-client, webcolors, uri-template, types-python-dateutil, tomli, tinycss2, tblib, send2trash, rfc3986-validator, rfc3339-validator, pywinpty, python-json-logger, pycparser, pandocfilters, overrides, mistune, locket, jupyterlab_widgets, jupyterlab-pygments, jsonpointer, json5, httpcore, fqdn, dataframe-api-compat, bleach, babel, async-lru, terminado, partd, cffi, arrow, modin, jupyter-server-terminals, isoduration, ipywidgets, httpx, dask, argon2-cffi-bindings, nbformat, jupyter-console, distributed, argon2-cffi, nbclient, jupyter-events, nbconvert, jupyter-server, notebook-shim, jupyterlab-server, jupyter-lsp, jupyterlab, notebook, jupyter, modin-spreadsheet

In [2]:
data = pd.read_excel('train_data_900k.xlsx')

In [3]:
import pandas as pd

In [4]:
data = pd.read_excel('train_data_900k.xlsx')